# Exploring Merged File

In [2]:
#Dependencies
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime as dt

In [28]:
path = os.getcwd()
#change path to datasets folder
os.chdir("../datasets/")
#read merged.pkl
merged_df = pd.read_pickle("./serialized/merged.pkl")

In [29]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 814242
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   calendar_day              814243 non-null  datetime64[ns]
 1   sku                       814243 non-null  object        
 2   size                      811071 non-null  object        
 3   block                     807619 non-null  object        
 4   ph                        813878 non-null  object        
 5   IN_material_description   774593 non-null  object        
 6   IN_material_type          774593 non-null  object        
 7   IN_strategy               767339 non-null  category      
 8   IN_shape                  771786 non-null  object        
 9   IN_steel_grade            771786 non-null  object        
 10  IN_length_text            769268 non-null  object        
 11  IN_tons                   774593 non-null  float64       
 12  SO

In [160]:
merged_df[merged_df["size"].str.contains("8 X 6",na=False)]

,calendar_day,sku,size,block,ph,IN_material_description,IN_material_type,IN_strategy,IN_shape,IN_steel_grade,IN_length_text,IN_tons,SO_order_qty_sales_units,SO_confirmed_qty,SH_shipment_tons,PR_ton
308,2016-01-03,109004183,5/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,"F 5/8X6 A36/44W/A572-50 20'00""",ZERT,MTO,FLAT,A36/44W/A572-50,"20'00""",0.003,NaN,NaN,NaN,NaN
316,2016-01-03,109004293,3/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,"F 3/8X6 50W 20'00""",ZERT,MTO,FLAT,50W,"20'00""",0.002,NaN,NaN,NaN,NaN
333,2016-01-03,109008152,3/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,"F 3/8X6 GGMULTI 20'00""",ZERT,MTS,FLAT,GGMULTI,"20'00""",326.468,NaN,NaN,NaN,NaN
341,2016-01-03,109008525,5/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,"F 5/8X6 GGMULTI 20'00""",ZERT,MTS,FLAT,GGMULTI,"20'00""",28.435,NaN,NaN,NaN,NaN
688,2016-01-04,109004183,5/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,"F 5/8X6 A36/44W/A572-50 20'00""",ZERT,MTO,FLAT,A36/44W/A572-50,"20'00""",0.003,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813599,2018-12-10,109008497,5/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0
813600,2018-12-10,109009166,5/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0
813602,2018-12-11,109008129,3/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0
813603,2018-12-11,109008151,3/8 X 6,"6"" FLT",LS-Mills-Mer-Mer-MBQ-MBQ Fl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0


In [154]:
merged_df[merged_df["block"]=='3-1/2X2-1/2"ANG']["size"].value_counts()

3 1/2X2 1/2X1/4     4601
3 1/2X2 1/2X3/8     3142
3 1/2X2 1/2X5/16    2251
3 1/2X2 1/2X1/2     1988
Name: size, dtype: int64

In [143]:
merged_df[merged_df["block"].str.contains('16', na=False)]["size"].value_counts()

#5 (16MM)     612
.678"         173
9/16"          17
3/16 X 3/4     12
15/16"          6
Name: size, dtype: int64

In [33]:
# look at block level
merged_df[merged_df["block"].isna()].groupby('sku').sum().max()

IN_tons                     632198.865803
SO_order_qty_sales_units     43562.941500
SO_confirmed_qty             43283.739500
SH_shipment_tons             47590.000000
PR_ton                       45707.000000
dtype: float64

In [35]:
#look at the size level
merged_df[merged_df["size"].isna()].groupby('sku').sum().max()

IN_tons                       28.25
SO_order_qty_sales_units    5964.32
SO_confirmed_qty            4097.90
SH_shipment_tons              95.00
PR_ton                        95.00
dtype: float64

In [36]:
merged_df[merged_df["size"].isna()]

,calendar_day,sku,size,block,ph,IN_material_description,IN_material_type,IN_strategy,IN_shape,IN_steel_grade,IN_length_text,IN_tons,SO_order_qty_sales_units,SO_confirmed_qty,SH_shipment_tons,PR_ton
89,2016-01-03,106022528,NaN,"2-1/2"" ANG",LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,L 2 1/2X2 1/2X3/16 A36/44W/A529-50 20'00,ZERT,MTO,NaN,NaN,NaN,0.025,NaN,NaN,NaN,NaN
469,2016-01-04,106022528,NaN,"2-1/2"" ANG",LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,L 2 1/2X2 1/2X3/16 A36/44W/A529-50 20'00,ZERT,MTO,NaN,NaN,NaN,0.025,NaN,NaN,NaN,NaN
849,2016-01-05,106022528,NaN,"2-1/2"" ANG",LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,L 2 1/2X2 1/2X3/16 A36/44W/A529-50 20'00,ZERT,MTO,NaN,NaN,NaN,0.025,NaN,NaN,NaN,NaN
1230,2016-01-06,106022528,NaN,"2-1/2"" ANG",LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,L 2 1/2X2 1/2X3/16 A36/44W/A529-50 20'00,ZERT,MTO,NaN,NaN,NaN,0.025,NaN,NaN,NaN,NaN
1614,2016-01-07,106022528,NaN,"2-1/2"" ANG",LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,L 2 1/2X2 1/2X3/16 A36/44W/A529-50 20'00,ZERT,MTO,NaN,NaN,NaN,0.025,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811463,2020-02-19,106022561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
811531,2020-02-21,110001842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
812210,2016-11-21,106028263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
813984,2019-08-01,106030348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0


In [161]:
os.getcwd()

'/Users/jacosta3/OneDrive - University of South Florida/Gerdau Projects/pft/datasets'

In [ ]:
#Convert to csv for easy share?
#merged_df.to_pickle(ser_path + "merged.pkl")